## Setup

Configuração do ambiente de desenvolvimento para realização do ETL (Extract Transform Load)

### Instalando bibliotecas

Instalando todas as bibliotecas necessárias e criação do ambiente spark

In [0]:
%sh
/databricks/python/bin/pip install pandas==2.2.2
/databricks/python/bin/pip install numpy==1.26.4
/databricks/python/bin/pip install pyarrow==7.0.0

# /databricks/python/bin/pip install pandas==2.2.3
/databricks/python/bin/pip install openpyxl==3.1.5
# /databricks/python/bin/pip install numpy==2.1.3
/databricks/python/bin/pip install requests==2.32.3
# /databricks/python/bin/pip install pyarrow==19.0.0



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


  Using cached pyarrow-7.0.0.tar.gz (816 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [614 lines of output]
      <string>:36: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      /tmp/pip-build-env-v_3bowkd/overlay/lib/python3.11/site-packages/setuptools/_distutils/dist.py:289: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      /tmp/pip-build-env-v_3bowkd/overlay/lib/python3.11/site-packages/setuptools/_distutils/dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      WARNING setuptools_scm.pyproject_reading toml section missing 'pyproject.toml does not contain a tool.setuptools_scm section'
      Traceback (most recent call last):
        File "/tmp/pip-build-env-v_3bowkd/overlay/lib/python3.11/site-packages/setuptools_scm/_integration/pyproject_reading.py", line 36, in read_pyp

Failed to build pyarrow
  Obtaining dependency information for openpyxl==3.1.5 from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


  Obtaining dependency information for requests==2.32.3 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [0]:
import os
import requests
import shutil
from zipfile import ZipFile

# from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import numpy as np

# Create a Spark session
# spark = SparkSession.builder.master("local").config("spark.executor.memory", "6g") \
#     .config("spark.driver.memory", "6g") \
#     .config("spark.driver.maxResultSize", "6g") \
#     .appName("PySpark Tutorial").getOrCreate()

# Verify Spark version
# print("Spark version: ", spark.version)

BASE_DOWNLOAD_URL_MICRO_DATA = 'https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_{month}{year}.zip'
BASE_DOWNLOAD_URL_GLOSSARY = 'https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Documentacao/Dicionario_PNAD_COVID_{month}{year}_20220621.xls'

BASE_PATH = '/Volumes/fiap_postech_covid19_pnad'

INPUT_PATH = f'{BASE_PATH}/ingest/ingest_raw/micro_data'
SILVER_PATH = f'{BASE_PATH}/silver/silver_volume'
BRONZE_PATH = f'{BASE_PATH}/bronze/bronze_raw'
BASE_DOWNLOAD_PATH = f'{BASE_PATH}/ingest/ingest_raw'

LIST_AVAILABLE_MONTHS = ['05', '06', '07', '08', '09', '10', '11']
LIST_AVAILABLE_YEARS = ['2020']

## Download

Download e organização das pastas em formato que possam ser utilizadas como partições spark


In [0]:
%sh

ls -lsah /Volumes/fiap_postech_covid19_pnad/bronze/bronze_raw/*/*/*

/Volumes/fiap_postech_covid19_pnad/bronze/bronze_raw/micro_data/ano_part=2020/mes=05:
total 84M
4.0K drwxrwxrwx 2 nobody nogroup 4.0K May 22 04:11 .
4.0K drwxrwxrwx 2 nobody nogroup 4.0K May 22 04:12 ..
 84M -rwxrwxrwx 1 nobody nogroup  84M May 22 04:11 PNAD_COVID_052020.csv

/Volumes/fiap_postech_covid19_pnad/bronze/bronze_raw/micro_data/ano_part=2020/mes=06:
total 92M
4.0K drwxrwxrwx 2 nobody nogroup 4.0K May 22 04:11 .
4.0K drwxrwxrwx 2 nobody nogroup 4.0K May 22 04:12 ..
 92M -rwxrwxrwx 1 nobody nogroup  92M May 22 04:11 PNAD_COVID_062020.csv

/Volumes/fiap_postech_covid19_pnad/bronze/bronze_raw/micro_data/ano_part=2020/mes=07:
total 110M
4.0K drwxrwxrwx 2 nobody nogroup 4.0K May 22 04:11 .
4.0K drwxrwxrwx 2 nobody nogroup 4.0K May 22 04:12 ..
110M -rwxrwxrwx 1 nobody nogroup 110M May 22 04:11 PNAD_COVID_072020.csv

/Volumes/fiap_postech_covid19_pnad/bronze/bronze_raw/micro_data/ano_part=2020/mes=08:
total 110M
4.0K drwxrwxrwx 2 nobody nogroup 4.0K May 22 04:11 .
4.0K drwxrwxrwx 2 

In [0]:
def download_file(url, path):
    local_filename = path
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename


def extract_file(zip_path, path_extract):
    ZipFile(zip_path).extractall(path_extract)

def process_micro_data(prefix_year, prefix_month, extract_path_name):

    base_download_path_zip = os.path.join(BASE_DOWNLOAD_PATH, 'zip')

    for year in LIST_AVAILABLE_YEARS:
        for month in LIST_AVAILABLE_MONTHS:
            url_download = BASE_DOWNLOAD_URL_MICRO_DATA.format(year=year, month=month)
            path_download = os.path.join(base_download_path_zip, f'{prefix_year}={year}', f'{prefix_month}={month}')
            path_extract = os.path.join(BRONZE_PATH, extract_path_name, f'{prefix_year}={year}', f'{prefix_month}={month}')
            file_path_download = os.path.join(path_download, f'{month}-{year}.zip')
            if not os.path.isdir(path_download): os.makedirs(path_download)
            if not os.path.isdir(path_extract): os.makedirs(path_extract)
            download_file(url_download, file_path_download)
            extract_file(file_path_download, path_extract)

    shutil.rmtree(base_download_path_zip)

def process_glossary_data():
    path_extract = os.path.join(BASE_DOWNLOAD_PATH, 'dicionario_pnad_covid_raw.xlsx')
    url_download = BASE_DOWNLOAD_URL_GLOSSARY.format(year='2020', month='11') # Usando ultimo mes, uma vez que é o dicionario mais completo
    download_file(url_download, path_extract)

prefix_year = 'ano_part'
prefix_month = 'mes'
extract_path_name = 'micro_data'

process_micro_data(prefix_year, prefix_month, extract_path_name)
process_glossary_data()

## Pre Processamento

Uma vez que os arquivos bases são grandes, para evitar esforço computacional e consumo de disco desnecessário, Estamos reescrevendo os dados Bronze para parquet, com somente os meses necessários (Necessário rodar somente uma vez, para gerar `bronze.parquet`)

In [0]:
# df = spark.read.csv(INPUT_PATH, header=True)

# df.write.format('parquet').mode("overwrite").partitionBy(['ano_part', 'mes']).save(BRONZE_PATH)


# df = spark.read.parquet(BRONZE_PATH, header=True)
# df.where(col('mes') >= 9).count()

---------------------------------------------------------------------------
PermissionError                           Traceback (most recent call last)
File <command-1905483557337270>, line 40
     37 prefix_month = 'mes'
     38 extract_path_name = 'micro_data'
---> 40 process_micro_data(prefix_year, prefix_month, extract_path_name)
     41 process_glossary_data()

File <command-1905483557337270>, line 24, in process_micro_data(prefix_year, prefix_month, extract_path_name)
     22 path_extract = os.path.join(BRONZE_PATH, extract_path_name, f'{prefix_year}={year}', f'{prefix_month}={month}')
     23 file_path_download = os.path.join(path_download, f'{month}-{year}.zip')
---> 24 if not os.path.isdir(path_download): os.makedirs(path_download)
     25 if not os.path.isdir(path_extract): os.makedirs(path_extract)
     26 download_file(url_download, file_path_download)

File <frozen os>:215, in makedirs(name, mode, exist_ok)

File <frozen os>:215, in makedirs(name, mode, exist_ok)

File <fr

In [0]:
# df = spark.read.parquet(BRONZE_PATH, header=True)
# df.count()


# df.where(col('mes') >= 9).count()

---------------------------------------------------------------------------
PermissionError                           Traceback (most recent call last)
File <command-1905483557337270>, line 40
     37 prefix_month = 'mes'
     38 extract_path_name = 'micro_data'
---> 40 process_micro_data(prefix_year, prefix_month, extract_path_name)
     41 process_glossary_data()

File <command-1905483557337270>, line 24, in process_micro_data(prefix_year, prefix_month, extract_path_name)
     22 path_extract = os.path.join(BRONZE_PATH, extract_path_name, f'{prefix_year}={year}', f'{prefix_month}={month}')
     23 file_path_download = os.path.join(path_download, f'{month}-{year}.zip')
---> 24 if not os.path.isdir(path_download): os.makedirs(path_download)
     25 if not os.path.isdir(path_extract): os.makedirs(path_extract)
     26 download_file(url_download, file_path_download)

File <frozen os>:215, in makedirs(name, mode, exist_ok)

File <frozen os>:215, in makedirs(name, mode, exist_ok)

File <fr